In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5, ProgressBar
import numpy as np
import itertools
import time
import matplotlib.pyplot as plt
import analysis.switching as sw
from adapt import refine
from analysis.h5shell import h5shell

In [3]:
from experiments.Pulse_Switching_CSHE import SwitchingExperiment

In [4]:
exp = SwitchingExperiment()
exp.sample = "CSHE2-C4R3"
exp.comment = "Phase Diagram -  P to AP - Interations = 5 - Settle delay = 50us - Reset 0.75V 6ns"
exp.polarity = -1 # 1: AP to P; -1: P to AP
exp.iteration = 5
exp.field.value = 0.0133
exp.measure_current = 3e-6
exp.reset_amplitude = 0.75
exp.reset_duration  = 6.0e-9
exp.settle_delay    = 50e-6
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die2-C4R3\CSHE2-C4R3-P2AP_2016-07-05.h5")
pbar = ProgressBar(num=1, notebook=True)
edges = [(exp.daq_buffer, wr.data), (exp.daq_buffer, pbar.data)]
# edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
coarse_ts = 1e-9*np.linspace(0.1, 5, 10) # List of durations
coarse_vs = np.linspace(0.4, 0.8, 10)
points    = [coarse_ts, coarse_vs]
points    = list(itertools.product(*points))

main_sweep = exp.add_unstructured_sweep([exp.pulse_duration, exp.pulse_voltage], points)

In [7]:
figs = []
t1 = time.time()
for i in range(exp.iteration):
    exp.reset()
    exp.run_sweeps()
    points, mean = sw.load_switching_data(wr.filename)
    figs.append(sw.phase_diagram_mesh(points, mean, title="Iteration={}".format(i)))
    new_points = refine.refine_scalar_field(points, mean, all_points=False,
                                criterion="integral", threshold = "one_sigma")
    if new_points is None:
        print("No more points can be added.")
        break
    #
    print("Elapsed time: {} min.".format((time.time()-t1)/60))
    main_sweep.update_values(new_points)

t2 = time.time()
exp.shutdown_instruments()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


15 new points added.
Elapsed time: 0.8608402689297994 min.

31 new points added.
Elapsed time: 1.0128604809443156 min.

53 new points added.
Elapsed time: 1.29448610941569 min.

73 new points added.
Elapsed time: 1.7651567419370016 min.

115 new points added.
Elapsed time: 2.3959474563598633 min.


In [8]:
print("Total points: {}".format(len(points)))
print("Total elapsed time: {} min.".format((t2-t1)/60))

Total points: 272
Total elapsed time: 2.3959474563598633 min.


In [9]:
plt.show()

In [10]:
points, mean = sw.load_switching_data(wr.filename, start_state=0)
fig_sw = sw.phase_diagram_mesh(points, mean)

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [13]:
# Reset failures
points, fails = sw.load_switching_data(wr.filename, failure=True)
fig_fail = sw.phase_diagram_mesh(points, fails)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [10]:
# For evaluation of adaptive method, plot the mesh
mesh, scale_factors = sw.scaled_Delaunay(points)
fig_mesh = sw.phase_diagram_mesh(points, mean, shading='gouraud')
plt.triplot(mesh.points[:,0]/scale_factors[0],
            mesh.points[:,1]/scale_factors[1], mesh.simplices.copy());